In [8]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
DATA_PATH = "data441_1.json"


def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    # Encode Labels
    labelencoder = LabelEncoder()
    labelencoder.fit(y)
    classes_num = labelencoder.transform(y)

    #OneHotEncoding
    encoder=OneHotEncoder(sparse=False, categories="auto")
    y=encoder.fit_transform(classes_num.reshape(len(classes_num),1))
    return X, y


def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs
        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()


def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.
    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split
    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X, y = load_data(DATA_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    return X_train, X_validation, X_test, y_train, y_validation, y_test


def build_model(input_shape):
    """Generates RNN-LSTM model
    :param input_shape (tuple): Shape of input set
    :return model: RNN-LSTM model
    """

    # build network topology
    model = keras.Sequential()

    # 2 LSTM layers
    model.add(keras.layers.LSTM(256, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.LSTM(256, return_sequences=True))
    model.add(keras.layers.Dropout(0.3))
    
    model.add(keras.layers.LSTM(256))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.3))
    # dense layer
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.3)) 

    # output layer
    model.add(keras.layers.Dense(11, activation='sigmoid'))

    return model

In [9]:
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.1, 0.15)

In [11]:
input_shape = (X_train.shape[1], X_train.shape[2]) # 130, 13
model = build_model(input_shape)

In [13]:
X_train.shape

(5128, 259, 13)

In [14]:
model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [15]:
filepath = "./callback441/weights-improvementx2-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]  

In [16]:
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=128, epochs=200,callbacks=callbacks_list)

===] - 51s 10ms/sample - loss: 0.0277 - accuracy: 0.9905 - val_loss: 0.3528 - val_accuracy: 0.9209
Epoch 69/200
5128/5128 [==============================] - 51s 10ms/sample - loss: 0.0294 - accuracy: 0.9907 - val_loss: 0.3984 - val_accuracy: 0.9202
Epoch 70/200
5128/5128 [==============================] - 51s 10ms/sample - loss: 0.0278 - accuracy: 0.9905 - val_loss: 0.3535 - val_accuracy: 0.9271
Epoch 71/200
5128/5128 [==============================] - 51s 10ms/sample - loss: 0.0253 - accuracy: 0.9910 - val_loss: 0.3160 - val_accuracy: 0.9311
Epoch 72/200
5128/5128 [==============================] - 51s 10ms/sample - loss: 0.0243 - accuracy: 0.9917 - val_loss: 0.4699 - val_accuracy: 0.9129
Epoch 73/200
5128/5128 [==============================] - 51s 10ms/sample - loss: 0.0295 - accuracy: 0.9904 - val_loss: 0.3174 - val_accuracy: 0.9263
Epoch 74/200
5128/5128 [==============================] - 51s 10ms/sample - loss: 0.0264 - accuracy: 0.9909 - val_loss: 0.3403 - val_accuracy: 0.9225
E

In [17]:
model.save('model441x2.h5')